In [5]:
import cv2
import numpy as np

def detect_lane(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply Gaussian blur to the image
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection to the image
    edges = cv2.Canny(blur, 50, 150)

    # Define the region of interest
    height, width = img.shape[:2]
    mask = np.zeros_like(edges)
    roi = np.array([[(0, height), (width/2, height/2), (width, height)]], dtype=np.int32)
    cv2.fillPoly(mask, roi, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define parameters for Hough transform
    rho = 1
    theta = np.pi/180
    threshold = 50
    min_line_length = 100
    max_line_gap = 50

    # Apply Hough transform to the image
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    # Fit the lanes with a polynomial curve model
    left_lane_pts = []
    right_lane_pts = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x1 == x2:
            continue
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        if slope < 0:
            left_lane_pts.append((x1, y1))
            left_lane_pts.append((x2, y2))
        else:
            right_lane_pts.append((x1, y1))
            right_lane_pts.append((x2, y2))
    left_lane_pts = np.array(left_lane_pts)
    right_lane_pts = np.array(right_lane_pts)
    left_curve = np.polyfit(left_lane_pts[:,0], left_lane_pts[:,1], 2)
    right_curve = np.polyfit(right_lane_pts[:,0], right_lane_pts[:,1], 2)

    # Draw the lanes on the image
    lane_img = np.zeros_like(img)
    yvals = np.linspace(0, height-1, height)
    left_fitx = left_curve[0]*yvals**2 + left_curve[1]*yvals + left_curve[2]
    right_fitx = right_curve[0]*yvals**2 + right_curve[1]*yvals + right_curve[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(lane_img, np.int_([pts]), (0,255,0))
    result = cv2.addWeighted(img, 1, lane_img, 0.3, 0)

    return result


In [9]:
img = cv2.imread('Curve.jpeg', 0)
imgs = detect_lane(img)
cv2.imshow('frame', imgs)

TypeError: 'NoneType' object is not iterable